In [ ]:
# default_exp shortest_path

# Shortest path

> subtitle tbd

edge weight = cost of a path


weights unequal --> use Djistra's algo

![](img/1.png)
![](img/2.png)

In [ ]:
# export
from queue import Queue
from typing import List, Dict
from collections import namedtuple

import pandas as pd

import graph_utils.core as gu

In [ ]:
g = gu.AdjacencyMatrixGraph(9, directed=True)
g.add_edge(0,1)
g.add_edge(1,2)
g.add_edge(2,7)
g.add_edge(2,4)
g.add_edge(2,3)
g.add_edge(1,5)
g.add_edge(5,6)
g.add_edge(3,6)
g.add_edge(3,4)
g.add_edge(6,8)
g.display()

0 --> 1
1 --> 2
1 --> 5
2 --> 3
2 --> 4
2 --> 7
3 --> 4
3 --> 6
5 --> 6
6 --> 8


In [ ]:
# export
Row = namedtuple('Node', ['distance_from_source', 'preceding_vertex'])

In [ ]:
?Row

Init signature: Row(distance_from_source, preceding_vertex)
Docstring:      Node(distance_from_source, preceding_vertex)
Type:           type
Subclasses:     


In [ ]:
# export
def _distance_table_to_df(distance_table:Dict):
    df = pd.DataFrame.from_dict(distance_table, orient='index')
    df.index.name = 'vertex_id'
    return df

In [ ]:
# export
def build_distance_table(graph:gu.Graph, source:int):
    distance_table = dict()

    # initiate an empty distance table
    for i in range(graph.numVertices):
        #distance_table[i] = Row(distance_from_source=None, preceding_vertex=None)
        distance_table[i] = Row(None, None)

    # distance to the source from itself is 0
    distance_table[source] = Row(0, source)

    queue = Queue()
    queue.put(source)

    while not queue.empty():
        current_vertex = queue.get()

        # the distance of current vertex from the source
        current_distance = distance_table[current_vertex][0] # zero is index of distance

        for neighbor in graph.get_adjacent_vertices(current_vertex):
            # only update dist table if no current distance from the source is set
            neigbor_was_visited = distance_table[neighbor][0] is not None
            if not neigbor_was_visited:
                distance_table[neighbor] = Row(current_distance + 1, current_vertex)

                # enqueue the neigbor only if in has other adjacent vertices to explore
                if len(graph.get_adjacent_vertices(neighbor)):
                    queue.put(neighbor)





    return distance_table

In [ ]:
build_distance_table(g, 0)


_distance_table_to_df(build_distance_table(g, 0))

,distance_from_source,preceding_vertex
vertex_id,,
0,0,0
1,1,0
2,2,1
3,3,2
4,3,2
5,2,1
6,3,5
7,3,2
8,4,6


In [ ]:
_distance_table_to_df(build_distance_table(g, 0))
expected_dict = {'distance_from_source': {0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 2, 6: 3, 7: 3, 8: 4},
                     'preceding_vertex': {0: 0, 1: 0, 2: 1, 3: 2, 4: 2, 5: 1, 6: 5, 7: 2, 8: 6}
           }

expected = pd.DataFrame(expected_dict)
got = _distance_table_to_df(build_distance_table(g, 0))
comparison_matrix = (got == expected)
not_equal = ~comparison_matrix.all(axis=1)

if not_equal.any():
    raise ValueError(f"{not_equal.sum()} vertices in distance table do not match")